# Projeto 2 - Ciência dos Dados

Nome: **André Barone Rocco**

Nome: **Rafael Meyer Kahn**

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [64]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
from IPython.display import display
import re
import emoji
import emoji_list

pd.options.display.max_rows = 20

___
## Autenticando no  Twitter

* Conta: ***@hayesfps***

In [2]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @hayesfps

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [3]:
#Produto escolhido:
produto = 'intel'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [4]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura (sem rt)
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
                break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [5]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens manualmente

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [65]:
#atribuindo arquivo a uma variável
dados = pd.read_excel('intel.xlsx')
dados_teste = pd.read_excel('intel.xlsx','Teste')

In [66]:
#exibindo Tweets
dados.head()

,Treinamento,Relevancia
0,@aziebert multithreaded command buffering é na...,0
1,ferramentas linux: como a equipe do clear linu...,0
2,"fique a conhecer o portátil de 14"" mais leve d...",0
3,@jornaloglobo a gente sabe que desde o primeir...,0
4,#workstation com dual #xeon 6240 (36-core/72-t...,0


In [67]:
dados_teste.head()

,Teste,Relevancia
0,@mxttewz intel vc me prometeu,0
1,#partiusp ... se os seus #servidores vão pra #...,0
2,"no entanto, quando falamos de gráficos, o amd ...",1
3,@apex_intel super mario galaxy 2,0
4,🎁‍super precio #amazon | #informática\nlenovo...,0


In [68]:
#funcao para limpar caracteres indesejados e para separar caracteres de palavras que poderiam estar juntos(emojis, '!', '?').
#necessário a instalação das livrarias 'emoji' e 'emoji_list'

def cleanup(tabela, titulo):
    
    a = tabela[titulo]
    a = a.str.lower()

    itens = ['.', ':', ';', '"', "'", '?', '(', ')', '[',']',',', '\n', '\t','*','|','+']
    i2 = ['?', '!']
    
    emojis = emoji_list.all_emoji
    
    #separa emojis de palavras
    for emoji in emojis:
        try:
            a = a.str.replace(emoji," {} ".format(emoji))
        except:
            pass
        
    #retira caracteres indesejados
    for e in itens:
        a = a.str.replace(e,'')
        
    #separa '?' e '!' de palavras
    for e in i2:
        a = a.str.replace(e,' {} '.format(e))

    #Substitui caracteres com acento ou variação por caracteres desejados
    a = a.str.replace('é','e')
    a = a.str.replace('ê','e')
    a = a.str.replace('á','a')
    a = a.str.replace('ã','a')
    a = a.str.replace('ô','o')
    a = a.str.replace('ó','o')
    a = a.str.replace('ú','u')
    a = a.str.replace('ç','c')
    a = a.str.replace('í','i')
    a = a.str.replace('@',' @')
    tabela[titulo] = a

    return tabela

In [69]:
#usando função para limpar caracteres indesejados da tabela de Tweets inicial
clean = cleanup(dados,'Treinamento')
clean.head()

,Treinamento,Relevancia
0,@aziebert multithreaded command buffering e n...,0
1,ferramentas linux como a equipe do clear linux...,0
2,fique a conhecer o portatil de 1 4 mais lev...,0
3,@jornaloglobo a gente sabe que desde o primei...,0
4,# workstation com dual # xeon 6 2 4 0 ...,0


In [70]:
#Criando tabela com apenas tweets irrelevantes
clean0 = clean.loc[(clean.Relevancia == 0)]
#Criando tabela com apenas tweets relevantes
clean1 = clean.loc[(clean.Relevancia == 1)]

In [71]:
#Função para separar palavras por tweet
def t_split(tabela, titulo):
    a = tabela[titulo].str.split(' ')

    li = pd.DataFrame()
    li['palavras'] = []

    for lista in a:
        for e in ['#','' , ' ']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if '@' in palavra and palavra != '@':
                lista.remove(palavra) 
            if palavra[:4] == 'http':
                lista.remove(palavra)
        li = li.append({'palavras': lista}, ignore_index=True)
                
    return li

In [72]:
#Separando tweets da tabela 'limpa'(relevantes)
splt1 = t_split(clean1,'Treinamento')
#Separando tweets da tabela 'limpa'(irrelevantes)
splt0 = t_split(clean0,'Treinamento')
splt1.head(10)

,palavras
0,"[intel, lanca, segunda, versao, de, seu, hyper..."
1,"[se, um, pc, e, intel, core, 3, sem, nenhum, p..."
2,"[significado, das, letras, de, sufixo, dos, no..."
3,"[nao, recomendo, amd, nem, pro, meu, pior, ini..."
4,"[o, intel, eh, o, modelo, mais, famoso, do, me..."
5,"[como, rodar, cyberpunk, 2, 0, 7, 7, num, inte..."
6,"[ja, nao, bastasse, os, bugs, de, processador,..."
7,"[se, eu, investisse, no, mercado, americano, c..."
8,"[@dantemenelau, comeco, da, grande, revolucao,..."
9,"[apple, ta, querendo, ser, igual, a, intel, to..."


In [73]:
#Função para criar tabela com palavras separadas individualmente a partir da tabela
def palavra_linha(tabela):
    a = tabela['palavras']
    prob = pd.DataFrame()
    prob['palavras'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'palavras': e}, ignore_index=True)
    
    return prob['palavras']

In [74]:
#Separando palavras individualmente(irrelevantes)
tab0 = palavra_linha(splt0)
#Separando palavras individualmente(relevantes)
tab1 = palavra_linha(splt1)
tab1

0            intel
1            lanca
2          segunda
3           versao
4               de
5              seu
6       hypervisor
7               de
8            nuvem
9          escrito
           ...    
1939         intel
1940         ultra
1941            hd
1942             2
1943             0
1944             d
1945           ate
1946        estage
1947           vai
1948           ter
Name: palavras, Length: 1949, dtype: object

In [75]:
#Frequências absolutas(relevantes)
frq1 = tab1.value_counts()
frq1.head()

e        80
intel    73
de       57
0        57
o        39
Name: palavras, dtype: int64

In [76]:
#Frequências absolutas(irrelevantes)
frq0 = tab0.value_counts()
frq0.head()

0    536
1    393
9    335
5    319
2    293
Name: palavras, dtype: int64

In [77]:
#Frequências relativas(relevantes)
frq_rel1 = tab1.value_counts(True)
#Exibindo frequências relativas em porcentagem
frq_rel1.head()*100

e        4.104669
intel    3.745511
de       2.924577
0        2.924577
o        2.001026
Name: palavras, dtype: float64

___
### Verificando a performance
​
Agora você deve testar o seu classificador com a base de Testes.


In [91]:
#criando coluna para resultados do classificador
dados_teste['Bot'] = 0

#criando coluna para comparação dos resultados do classificador Vs feitos à mão 
dados_teste['Resultado'] = pd.Series()

#mudando DataFrame de teste para que classificador possa ler
teste_clean = cleanup(dados_teste,'Teste')
teste_f = t_split(teste_clean,'Teste')

#função para multiplicar itens dentro de uma lista
def multiplica(Lista) :     
    u = 1
    for x in Lista: 
         u = u * x  
    return u  


In [82]:
i = 0
for lin in teste_f['palavras']:
    p0 = 1
    p1 = 1
    prob1 = []
    prob0 = []
    for palavra in lin:
        try:
            p1 *= (frq1[palavra] +1)/(sum(frq1) + len(frq1))
            p0 *= (frq0[palavra] +1)/(sum(frq0) + len(frq0))
            prob1.append(p1)
            prob0.append(p0)
        except:
            p1 *= 1/(sum(frq1) + len(frq1))
            p0 *= 1/(sum(frq0) + len(frq0))
            prob1.append(p1)
            prob0.append(p0)
            
    multiplica1 = multiplica(prob1)   
    multiplica0 = multiplica(prob0)
    
    if multiplica1 > multiplica0:
        dados_teste['Bot'][i] = 1
        if dados_teste['Bot'][i] == dados_teste['Relevancia'][i]:
            dados_teste['Resultado'][i] = 'Positivo Verdadeiro'
        elif dados_teste['Bot'][i] != dados_teste['Relevancia'][i]:
            dados_teste['Resultado'][i] = 'Positivo Falso'
    else:
    #elif multiplica0 > multiplica1:
        dados_teste['Bot'][i] = 0
        if dados_teste['Bot'][i] == dados_teste['Relevancia'][i]:
            dados_teste['Resultado'][i] = 'Negativo Verdadeiro'
        elif dados_teste['Bot'][i] != dados_teste['Relevancia'][i]:
            dados_teste['Resultado'][i] = 'Negativo Falso'
    i += 1

C:\Users\Aspire_andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Aspire_andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Aspire_andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Aspire_andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set o

In [88]:
#Visualização da tabela pronta:
dados_teste

,Teste,Relevancia,Bot,Resultado
0,@mxttewz intel vc me prometeu,0,1,Positivo Falso
1,# partiusp se os seus # servidores vao pra ...,0,0,Negativo Verdadeiro
2,no entanto quando falamos de graficos o amd sa...,1,0,Negativo Falso
3,@apex_intel super mario galaxy 2,0,1,Positivo Falso
4,🎁 ‍ super precio # amazon # informatical...,0,0,Negativo Verdadeiro
5,olimpiadas de 2 0 2 0 terao reconheciment...,0,0,Negativo Verdadeiro
6,servidores dual xeon com apenas 1 processado...,0,0,Negativo Verdadeiro
7,olimpiada de toquio tera campeonato de street ...,0,0,Negativo Verdadeiro
8,@memeshltv intel extreme masters santa efigenia,0,1,Positivo Falso
9,no magazine luiza voce encontranotebook dell i...,0,0,Negativo Verdadeiro


In [89]:
#Frequencia relativa do resultado das comparações
dados_teste['Resultado'].value_counts(True)*100

Negativo Verdadeiro    63.0
Positivo Falso         15.0
Negativo Falso         15.0
Positivo Verdadeiro     7.0
Name: Resultado, dtype: float64

In [93]:
dados_teste['Relevancia'].value_counts(True)*100

0    78.0
1    22.0
Name: Relevancia, dtype: float64

### Perguntas para o aperfeiçoamento

#### Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
É possível imaginar outros cenários para o classificador de Naive Bayes além da classificação de Tweets, como por exemplo prever avaliações de filmes e séries, através de análises anteriores de usuários e pessoas de referência na área, fazendo com que assim seus produtores tenham ideia do que esperar da crítica de suas produções. Outro exemplo da aplicação desse classificador é a previsão da aceitação de videogames, que ao se analisar o sucesso de diversos jogos similares e de mecânicas que estão presentes nesses jogos, torna possível estimar se o jogo irá fazer sucesso ou não no meio que ele será inserido.
#### Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
Para tornar o classificador mais preciso e confiável, seria necessário fornecê-lo uma base de dados maior, fazendo com que o filtro ficasse melhor devido a maior quantidade de exemplos para analisar e determinar palavras chave de exemplos relevantes ou irrelevantes.
Outro exemplo de melhoria seria a implementação de mais classificações para o programa, como por exemplo Tweets Muito relevantes, Críticas, Elogios, entre outras, para assim poder ter mais precisão ainda no tipo de tweet que a empresa deseja analisar. Isso proporciona uma economia de tempo quando precisa-se cuidar da imagem que seu produto passa aos consumidores usuários de redes sociais, dando a possibilidade de responder os tweets importantes. Por fim, outra grande melhoria seria criar uma nova classificação relacionada ao assunto do tweet a ser respondido, como por exemplo para uma desenvolvedora de jogos, que ao separar os tweets pelos jogos que eles estão falando, é possível manda-los corretamente para cada área da desenvolvedora, para assim saber as melhorias que os usuários querem para cada jogo que estão utilizando.


___
### Concluindo

Em suma, a partir dos resultados obtidos com o uso do classificador, é possível notar que o número de Positivos Verdadeiros(7%),tweets que foram dados como relevantes e realmente são relevantes, é compensado pela grande efetividade do classificador de Naive Bayes em obter Negativos Verdadeiros(63%),tweets classificados como não relevante e que de fato não são relevantes, fator este que comprova sua eficácia em filtrar Tweets que uma empresa deveria prestar atenção quando for utilizar suas redes sociais para fazer pesquisas e analisar a satisfação de seus consumidores. Também é necessário levar em conta que, embora 7% seja baixo quando comparado ao total, a porcentagem de Tweets relevantes avaliados manualmente era de 22%. Este é o real valor máximo de Tweets relevantes

Portanto, para tornar o classificador mais preciso e confiável, seria necessário fornecê-lo uma base de dados maior, fazendo com que o filtro ficasse melhor devido a maior quantidade de exemplos para analisar e determinar palavras chave de exemplos relevantes ou irrelevantes.

Outro exemplo de melhoria, seria a implementação de mais classificações para o programa, como por exemplo Tweets Muito relevantes, Críticas, Elogios,  Muito irrelevantes, entre outras, para assim poder ter mais precisão ainda no tipo de tweet que a empresa deseja responder, o que proporciona uma economia de tempo quando a empresa precisa cuidar de suas redes sociais, dando a possibilidade de responder os tweets importantes invés de ter que ficar procurando por eles por um longo tempo.


Logo, com tudo o que foi abordado durante está conclusão, é possível afirmar que o classificador de Naive Bayes é uma ferramenta extremamente eficaz e poderosa para fazer a classificação de vários dados, não ficando apenas no cenário das redes sociais como o Twiter, mas podendo ser utilizado amplamente em diversas outras áreas fora do campo de  Machine Learning, o que faz com que o estudo de seu funcionamento e da maneira de seu desenvolvimento uma grande ajuda para a solução de diversos problemas.


## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**